In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import pandas as pd
import numpy as np
import sources
import os
import json


In [2]:
from src.config.attribute_dict import AttributeDict

In [3]:
cfg = sources.default_config()
mlclient = MlflowClient(cfg.mlflow.uri)

In [4]:
EXP = "SteeredCNN-AV"
exp = mlclient.get_experiment_by_name(EXP)


In [46]:
def parse_runs(runs):
    df_metrics = pd.DataFrame()
    df_params = pd.DataFrame()
    df_miscs = pd.DataFrame()
    df_runs = pd.DataFrame()
    
    for i, r in enumerate(runs):
        df_runs.at[i, 'exp'] = r.data.tags.get('exp', '-')
        df_runs.at[i, 'sub'] = r.data.tags.get('sub', '-')
        df_runs.at[i, 'subexp'] = r.data.tags.get('subexp', '-')
        
        for k, v in r.data.metrics.items():
            df_metrics.at[i, k] = v
            
        for k, v in r.data.params.items():
            df_params.at[i, ('params', k)] = v
            
        artifacts_path = r.info.artifact_uri.split('polymtl.ca')[1]
        try:
            with open(os.path.join(artifacts_path, 'misc.json'), 'r') as json_f:
                misc = AttributeDict.from_dict(json.load(json_f), recursive=True)
        except:
            pass
        else:
            for k in misc.walk():
                df_miscs.at[i, ('misc', k)] = misc[k]
    dataframes = {'run': df_runs, 'metrics': df_metrics, 'params': df_params, 'miscs': df_miscs}
    return pd.concat(dataframes.values(), axis=1, keys=dataframes.keys()).fillna('-')

def fetch_runs(exp, filter=None):
    if isinstance(exp, str):
        exp = mlclient.get_experiment_by_name(EXP)
    runs = mlclient.list_run_infos(exp.experiment_id, ViewType.ACTIVE_ONLY)
    runs = [mlclient.get_run(r.run_id) for r in runs if r.status=='FINISHED']
    
    if filter is not None:
        runs = [_ for _ in runs if filter(_)]
    
    return parse_runs(runs)

In [47]:
d = fetch_runs(exp, lambda r: r.data.tags.get('subexp', '-').startswith('[ParamStarve]'))
del d['params']
for m in list(d['metrics'].columns):
    if not m.startswith(('MESS','HRF','DRI')) and m!='best-val-acc-epoch':
        del d[('metrics', m)]
d

run                                                  metrics  \
             exp            sub                       subexp MESSIDOR-acc   
0    ParamStarve  2-2,4,8,16,32  [ParamStarve] 2-2,4,8,16,32     0.645731   
1    ParamStarve  2-2,4,8,16,32  [ParamStarve] 2-2,4,8,16,32     0.530840   
2    ParamStarve  2-2,4,8,16,32  [ParamStarve] 2-2,4,8,16,32     0.467482   
3    ParamStarve  2-2,4,8,16,32  [ParamStarve] 2-2,4,8,16,32     0.485920   
4    ParamStarve  2-2,4,8,16,32  [ParamStarve] 2-2,4,8,16,32     0.449294   
..           ...            ...                          ...          ...   
329  ParamStarve           3-22           [ParamStarve] 3-22     0.382979   
330  ParamStarve           3-22           [ParamStarve] 3-22     0.566114   
331  ParamStarve           3-22           [ParamStarve] 3-22     0.614115   
332  ParamStarve           3-22           [ParamStarve] 3-22     0.574256   
333  ParamStarve           3-22           [ParamStarve] 3-22     0.387489   

                                                                      \
    MESSIDOR-roc MESSIDOR-iou DRIVE-iou best-val-acc-epoch   HRF-acc   
0       0.742952     0.476760  0.632712               47.0  0.749103   
1       0.634040     0.356333  0.654897               28.0  0.700879   
2       0.631546     0.283961  0.550692               39.0  0.607082   
3       0.544999     0.317736  0.637790               39.0  0.689413   
4       0.620271     0.259718  0.557102               30.0  0.645279   
..           ...          ...       ...                ...       ...   
329     0.500000     0.191490  0.238965               13.0  0.479652   
330     0.664708     0.390936  0.683185               46.0  0.742016   
331     0.684567     0.440932  0.486479               40.0  0.682733   
332     0.661944     0.402629  0.730881               49.0  0.755480   
333     0.497088     0.201221  0.238366               21.0  0.481325   

                                                  miscs               
      HRF-roc   HRF-iou DRIVE-acc DRIVE-roc        misc model.params  
0    0.837694  0.597572  0.775433  0.858633    132441.0     132441.0  
1    0.789157  0.535258  0.791615  0.873450    132441.0     132441.0  
2    0.662749  0.409743  0.714144  0.782367    132441.0     132441.0  
3    0.775035  0.524360  0.778859  0.867876    132441.0     132441.0  
4    0.782377  0.453619  0.721155  0.819149    132441.0     132441.0  
..        ...       ...       ...       ...         ...          ...  
329  0.499900  0.239834  0.477929  0.500000  29327585.0   29327585.0  
330  0.812548  0.587452  0.811872  0.900223  29327585.0   29327585.0  
331  0.745262  0.518225  0.655893  0.714911  29327585.0   29327585.0  
332  0.846367  0.603219  0.844525  0.921816  29327585.0   29327585.0  
333  0.501501  0.243317  0.468182  0.490990  29327585.0   29327585.0  

[334 rows x 15 columns]

In [48]:
g = (d.sort_values(('metrics', 'DRIVE-acc'), ascending=False)
      .groupby(('run', 'subexp')).head(3)
      .groupby(('run', 'subexp')).agg(['mean','std'])
      .sort_values(('metrics','DRIVE-acc','mean'), ascending=False) )

/home/gaby/.conda/envs/nnet/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2878: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


(run, subexp)
[ParamStarve] 2-22             83.3 ± 0.3%
[ParamStarve] 2-6              83.2 ± 0.3%
[ParamStarve] 2-11             82.9 ± 0.3%
[ParamStarve] 1-11             82.5 ± 0.2%
[ParamStarve] 1-22             82.3 ± 0.0%
[ParamStarve] 1-6              81.1 ± 0.9%
[ParamStarve] 3-22             79.8 ± 5.4%
[ParamStarve] 3-11             79.7 ± 2.3%
[ParamStarve] 2-2              79.5 ± 0.1%
[ParamStarve] 2-2,4,8,16,32    79.4 ± 0.5%
[ParamStarve] 2-3              78.0 ± 1.4%
[ParamStarve] 3-6              77.1 ± 0.7%
[ParamStarve] 1-2              76.4 ± 1.0%
[ParamStarve] 3-3              75.1 ± 1.2%
[ParamStarve] 3-2              74.3 ± 0.8%
[ParamStarve] 2-2,4,6,8,8      73.7 ± 0.6%
[ParamStarve] 2-2,2,2,2,2      72.6 ± 0.3%
dtype: object

In [78]:
params = d.groupby(('run','subexp')).head(1)
params.index = params[('run', 'subexp')]
g['param'] = params[('miscs','model.params')]
g['top3-acc'] = g[('metrics','DRIVE-acc')].apply(lambda r: f"{r['mean']*100:.1f} ± {r['std']:.1%}", axis=1)

In [81]:
g[['param', 'top3-acc']]

,param,top3-acc
,,
,,
"(run, subexp)",,
[ParamStarve] 2-22,15941641.0,83.3 ± 0.3%
[ParamStarve] 2-6,1187401.0,83.2 ± 0.3%
[ParamStarve] 2-11,3987501.0,82.9 ± 0.3%
[ParamStarve] 1-11,640179.0,82.5 ± 0.2%
[ParamStarve] 1-22,2555697.0,82.3 ± 0.0%
[ParamStarve] 1-6,191089.0,81.1 ± 0.9%
[ParamStarve] 3-22,-,79.8 ± 5.4%


In [11]:
[2**_ for _ in range(6)]

[1, 2, 4, 8, 16, 32]

In [9]:
sum(_1*_2 for _1, _2 in zip([2,4,6,8,12], [4,4,4,4,2]))

104

In [95]:
np.logspace(np.sqrt(18),np.log2(184),5, base=2)

array([ 18.93050099,  33.42535562,  59.01874433, 104.20867986,
       184.        ])

In [42]:
d

metrics                                                         \
    MESSIDOR-acc MESSIDOR-roc MESSIDOR-iou DRIVE-iou best-val-acc-epoch   
0       0.645731     0.742952     0.476760  0.632712               47.0   
1       0.530840     0.634040     0.356333  0.654897               28.0   
2       0.467482     0.631546     0.283961  0.550692               39.0   
3       0.485920     0.544999     0.317736  0.637790               39.0   
4       0.449294     0.620271     0.259718  0.557102               30.0   
..           ...          ...          ...       ...                ...   
329     0.382979     0.500000     0.191490  0.238965               13.0   
330     0.566114     0.664708     0.390936  0.683185               46.0   
331     0.614115     0.684567     0.440932  0.486479               40.0   
332     0.574256     0.661944     0.402629  0.730881               49.0   
333     0.387489     0.497088     0.201221  0.238366               21.0   

                                                            miscs               
      HRF-acc   HRF-roc   HRF-iou DRIVE-acc DRIVE-roc        misc model.params  
0    0.749103  0.837694  0.597572  0.775433  0.858633    132441.0     132441.0  
1    0.700879  0.789157  0.535258  0.791615  0.873450    132441.0     132441.0  
2    0.607082  0.662749  0.409743  0.714144  0.782367    132441.0     132441.0  
3    0.689413  0.775035  0.524360  0.778859  0.867876    132441.0     132441.0  
4    0.645279  0.782377  0.453619  0.721155  0.819149    132441.0     132441.0  
..        ...       ...       ...       ...       ...         ...          ...  
329  0.479652  0.499900  0.239834  0.477929  0.500000  29327585.0   29327585.0  
330  0.742016  0.812548  0.587452  0.811872  0.900223  29327585.0   29327585.0  
331  0.682733  0.745262  0.518225  0.655893  0.714911  29327585.0   29327585.0  
332  0.755480  0.846367  0.603219  0.844525  0.921816  29327585.0   29327585.0  
333  0.481325  0.501501  0.243317  0.468182  0.490990  29327585.0   29327585.0  

[334 rows x 12 columns]